In [1]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

In [2]:
! rm ./test_db3.db

In [3]:
class Borehole(Base):
    """The Boreholes Info table"""
    __tablename__ = 'Boreholes'
    id = Column(String(32), primary_key=True)
    intervals = relationship(
        'Interval',
        collection_class=attribute_mapped_collection('id'),
        cascade='all, delete-orphan')
    intervals_values = association_proxy(
        'intervals', 'value',
        creator=lambda k, v: Interval(id=k, description=v['description'], interval_number=v['interval_number']))

class Interval(Base):
    """The Interval table"""
    __tablename__ = 'Intervals'
    id = Column(Integer, primary_key=True)
    borehole = Column(String(32), ForeignKey('Boreholes.id',ondelete="CASCADE", onupdate="CASCADE"))
    # Note that this could be a numeric ID as well
    interval_number = Column(Integer)
    components = relationship('Component',secondary='Linkintervalcomponent')
    description = Column(String(32))
    
class Component(Base):
    """The Component table"""
    __tablename__ = 'Components'
    id = Column(String(32), primary_key=True)
    intervals = relationship(Interval,secondary='Linkintervalcomponent')
    description = Column(String(32))

class LinkIntervalComponent(Base):
    __tablename__ = 'Linkintervalcomponent'

    int_id = Column(
        Integer, 
        ForeignKey('Intervals.id'), 
        primary_key = True)

    comp_id = Column(
       Integer, 
       ForeignKey('Components.id'), 
       primary_key = True)

In [4]:
engine = create_engine('sqlite:///test_db3.db', echo=True)

In [5]:
Base.metadata.create_all(engine)

2020-12-18 09:17:55,934 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-18 09:17:55,938 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 09:17:55,944 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-18 09:17:55,946 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 09:17:55,950 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-18 09:17:55,953 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 09:17:55,958 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-18 09:17:55,961 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 09:17:55,964 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Intervals")
2020-12-18 09:17:55,966 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 09:17:55,970 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Intervals")
2020-12-18 09:17:55,973 INFO sqlalchemy.engine.base.Engine ()
2020-12-18

In [6]:
from contextlib import contextmanager

Session = sessionmaker(bind=engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [7]:
from striplog import Lexicon, Striplog

In [8]:
lexicon = Lexicon.default()

In [9]:
borehole_dict = {'F01':'test.las', 'F02':'test.las'} # boreholes to insert into the db

In [10]:
int_id = 0
bh_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(Borehole(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number}})
        interval_number+=1
        int_id += 1
    # print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

In [11]:
component_dict

{Component({'lithology': 'siltstone', 'colour': 'grey'}): 0,
 Component({'lithology': 'anhydrite'}): 1,
 Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}): 2,
 Component({'lithology': 'dolomite'}): 3,
 Component({'lithology': 'siltstone', 'colour': 'red'}): 4,
 Component({'lithology': 'limestone'}): 5}

In [12]:
strip.components

[Component({'lithology': 'siltstone', 'colour': 'grey'}),
 Component({'lithology': 'anhydrite'}),
 Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}),
 Component({'lithology': 'dolomite'}),
 Component({'lithology': 'siltstone', 'colour': 'red'}),
 Component({'lithology': 'limestone'})]

In [13]:
strip[0].components

[Component({'lithology': 'anhydrite'})]

In [14]:
boreholes

[<__main__.Borehole at 0x7f19e40b9390>, <__main__.Borehole at 0x7f19e405e128>]

In [15]:
with session_scope() as session:
    for bh in boreholes:
        session.add(bh)

2020-12-18 09:17:59,526 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 09:17:59,531 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-18 09:17:59,534 INFO sqlalchemy.engine.base.Engine (('F01',), ('F02',))
2020-12-18 09:17:59,552 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description) VALUES (?, ?, ?, ?)
2020-12-18 09:17:59,557 INFO sqlalchemy.engine.base.Engine ((0, 'F01', 0, 'Anhydrite'), (1, 'F01', 1, 'Sandstone, grey, vf-f'), (2, 'F01', 2, 'Anhydrite'), (3, 'F01', 3, 'Dolomite'), (4, 'F01', 4, 'Anhydrite'), (5, 'F01', 5, 'Sandstone, grey, vf-f'), (6, 'F01', 6, 'Siltstone, red'), (7, 'F01', 7, 'Dolomite')  ... displaying 10 of 50 total bound parameter sets ...  (48, 'F02', 23, 'Limestone'), (49, 'F02', 24, 'Volcanic'))
2020-12-18 09:17:59,564 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
class Project:
    def __init__(self, session):
        self.session = session
        self.refresh()
        #self.boreholes = None

    def refresh(self):
        self.boreholes = self.session.query(Borehole).all()

        
    def commit(self):
        self.session.commit()
        
    def add_borehole(self, bh):
        self.session.add(bh)
        self.commit()
        self.refresh()

    def row_count(self):
        return len(self.boreholes)

    def write_db(self):
        pass
    
    def add_components(self, components):
        for id in components.keys():
            new_compo = Component(id=id, description=components[id].summary()) 
            self.session.add(new_compo)
        self.commit()
        self.refresh()
        

In [17]:
session = Session()
p = Project(session)

2020-12-18 09:17:59,883 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 09:17:59,885 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-18 09:17:59,887 INFO sqlalchemy.engine.base.Engine ()


In [18]:
p.boreholes[1].id='F33'

In [19]:
p.boreholes[1].id

'F33'

In [20]:
p.commit()

2020-12-18 09:18:00,133 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET id=? WHERE "Boreholes".id = ?
2020-12-18 09:18:00,137 INFO sqlalchemy.engine.base.Engine ('F33', 'F02')
2020-12-18 09:18:00,140 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
p.boreholes[0].id

2020-12-18 09:18:00,266 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 09:18:00,271 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2020-12-18 09:18:00,273 INFO sqlalchemy.engine.base.Engine ('F01',)


'F01'

In [22]:
p.boreholes[0].intervals

2020-12-18 09:18:00,345 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2020-12-18 09:18:00,348 INFO sqlalchemy.engine.base.Engine ('F01',)


{0: <__main__.Interval at 0x7f19e406ec18>,
 1: <__main__.Interval at 0x7f19e40b92b0>,
 2: <__main__.Interval at 0x7f19e40b9240>,
 3: <__main__.Interval at 0x7f19e40b91d0>,
 4: <__main__.Interval at 0x7f19e40b9128>,
 5: <__main__.Interval at 0x7f19e40b9080>,
 6: <__main__.Interval at 0x7f19e40b9358>,
 7: <__main__.Interval at 0x7f19e40b9438>,
 8: <__main__.Interval at 0x7f19e405e048>,
 9: <__main__.Interval at 0x7f19e405edd8>,
 10: <__main__.Interval at 0x7f19e405e080>,
 11: <__main__.Interval at 0x7f19e405efd0>,
 12: <__main__.Interval at 0x7f19e405ee80>,
 13: <__main__.Interval at 0x7f19e405e240>,
 14: <__main__.Interval at 0x7f19e405e470>,
 15: <__main__.Interval at 0x7f19e405e828>,
 16: <__main__.Interval at 0x7f19e405ec18>,
 17: <__main__.Interval at 0x7f19e405e390>,
 18: <__main__.Interval at 0x7f19e405e4e0>,
 19: <__main__.Interval at 0x7f19e405e668>,
 20: <__main__.Interval at 0x7f19e405e5f8>,
 21: <__main__.Interval at 0x7f19e405e5c0>,
 22: <__main__.Interval at 0x7f19e405e940>

In [23]:
p.add_borehole(Borehole(id='F35'))

2020-12-18 09:18:00,423 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-18 09:18:00,428 INFO sqlalchemy.engine.base.Engine ('F35',)
2020-12-18 09:18:00,435 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-18 09:18:00,575 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 09:18:00,579 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-18 09:18:00,582 INFO sqlalchemy.engine.base.Engine ()


In [24]:
p.add_components(components)

2020-12-18 09:18:00,608 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2020-12-18 09:18:00,613 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2020-12-18 09:18:00,621 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-18 09:18:00,841 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 09:18:00,843 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-18 09:18:00,847 INFO sqlalchemy.engine.base.Engine ()


In [25]:
bh = p.boreholes[0]

In [26]:
bh.intervals[0]

2020-12-18 09:18:00,956 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2020-12-18 09:18:00,969 INFO sqlalchemy.engine.base.Engine ('F01',)


In [27]:
p.boreholes[-1].id

'F35'